# Process walkthrough:
- Access an excel file through user defined path
- Loop through each sheet to look for unique identifiers like 'Group Name:' to identify Oxford Plans of the Table 1 format using condition given in the `collecting tables` function
- Once Table 1 format is detected, check if there are multiple tables present in the sheet.
- If there are more than one table in a sheet, use the `separate_table` function to separate the tables and store them individually in a list
- If there is only 1 table in the sheet, again, store it in the same list 
- Loop over each table in the list and clean the data
- Once cleaned, concatenate the clean dfs into a final df

## Separating tables and Collecting them individually

In [55]:
import pandas as pd

#function to separate tables in case of multiple tables in a sheet

def separate_tables(df, df_list):
    # Convert all columns to string type
    df = df.astype(str)
    
    # Check if 'Group Name:' row exists in the DataFrame
    if df.apply(lambda x: x.str.contains('Group Name:')).any().any():
        # Find the row index where 'Group Name:' is present
        header_index = (df.apply(lambda x: x.str.contains('Group Name:')).any(axis=1)).idxmax()

        # Split the DataFrame into df1 and df2 based on the 'Group Name:' row
        df1 = df.iloc[:header_index]
        df2 = df.iloc[header_index + 1:]

        # Set the header of df2 using the 'Group Name:' row
        df2.columns = df.iloc[header_index]

        # Reset index for both DataFrames
        df1 = df1.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)

        df_list.append(df1)
        df_list.append(df2)
        
    

#function to extract tables individually and store them in a list

def collecting_tables(file_path):
    # Read the Excel file
    xls = pd.ExcelFile(file_path)
    
    # Initialize an empty list to store dataframes
    table_list = []
    
    # Iterate through each sheet
    for sheet_name in xls.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(file_path, header=None, sheet_name=sheet_name)
        
        # Check if "Group Name:" exists in the top-left cells
        if "Group Name:" in df.iloc[:5, :5].values:
            df = df.T
            df.columns = df.iloc[0]
            df = df[1:]
            df.dropna(axis=0, how='all', inplace=True)
            df.reset_index(inplace=True, drop=True)
            
            df=df.astype(str)
            
            
            
            if (df.applymap(lambda x: 'Group Name:' in x)).any(axis=1).any():
                separate_tables(df, table_list)
            

                for table in table_list:
                    separate_tables(table, table_list)
                table_list = [item for item in table_list if not item.apply(lambda x: x.str.contains('Group Name:')).any().any()]
                
            else:
                table_list.append(df)
            
            
        else:
            pass
    
    
    return table_list


In [56]:
table_list=collecting_tables(r"C:\Users\Subhadeep\Downloads\multi table oxford sheet (1).xlsx")


C:\Users\SUBHAD~1\AppData\Local\Temp/ipykernel_4148/4097615813.py:45: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "Group Name:" in df.iloc[:5, :5].values:


In [57]:
len(table_list)

7

In [58]:
table_list[0]

,Group Name:,Group Number(or TBD),Request Date:,Effective Date or NSB:,Please Note: All items listed on this form are subject to review,NaN,NaN,General Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product,Access,Network,In Network (IN),PCP/Specialist OV Copay,ER Cost Share,Hospital Cost Share,"For Direct, EPO or Value Option Products",IN Deductible,IN Coinsurance %,IN Coinsurance Limit,IN Out of Pocket Maximum,Out of Network (OON):,OON Deductible,OON Coinsurance %,OON Coinsurance Limit,OON Out of Pocket Maximum,NaN
0,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,nan,nan,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,nan,23,NJ,Large,HMO,Non Gated,Freedom,nan,30/45,150,IP: 0 after deductible OP: 0 after deductible,nan,2500/5000,100,3000/6000,3000/6001,nan,2500/5002,92,32000/50002,32000/50002,nan
1,nan,nan,nan,nan,nan,nan,nan,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,nan,25,NJ,Large,HMO,Non Gated,Freedom,nan,30/50,160,IP: 0 after deductible OP: 0 after deductible,nan,2500/5000,90,32000/50000,32000/50000,nan,2500/5003,93,32000/50003,32000/50003,nan
2,nan,nan,nan,nan,nan,nan,nan,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,nan,45,NJ,Large,HMO,Non Gated,Freedom,nan,0/0,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5001,91,32000/50001,32000/50001,nan,2500/5004,94,32000/50004,32000/50004,nan
3,nan,nan,nan,nan,nan,nan,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,nan,30/55,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5002,92,32000/50002,32000/50002,nan,2500/5002,92,32000/50002,32000/50002,nan
4,nan,nan,nan,nan,nan,nan,nan,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,nan,30/60,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5003,93,32000/50003,32000/50003,nan,2500/5003,93,32000/50003,32000/50003,nan
5,nan,nan,nan,nan,nan,nan,nan,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,nan,0/1,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5004,94,32000/50004,32000/50004,nan,2500/5004,94,32000/50004,32000/50004,nan


## Cleaning the data

In [49]:
import pandas as pd
import numpy as np
def clean_data(df_list):
    dfs = []
    
    for df in df_list:
        df.replace('nan', np.nan, inplace=True)
        df['Group Name:']=df['Group Name:'].fillna(df['Group Name:'].iloc[0])
        df['Group Number(or TBD)']=df['Group Number(or TBD)'].fillna(df['Group Number(or TBD)'].iloc[0])
        df['Request Date:']=df['Request Date:'].fillna(df['Request Date:'].iloc[0])
        df['Effective Date or NSB:']=df['Effective Date or NSB:'].fillna(df['Effective Date or NSB:'].iloc[0])
        
        
        # Rename columns to strings
        df.columns = [str(column) for column in df.columns]
        
        # Iterate through each column to check for 'Requested Benefits' in string columns
        for column in df.select_dtypes(include='object').columns:
            if any(df[column].apply(lambda x: 'Requested Benefits' in str(x))):
                df.rename(columns={column: 'Benefits Status'}, inplace=True)

        # Fill missing values in 'Benefits Status' column with forward fill
        df['Benefits Status']=df['Benefits Status'].fillna(method='ffill')

        # Rename 'General Information' column to 'Plan Information'
        df.rename(columns={'General Information': 'Plan Information'}, inplace=True)
        df.dropna(axis=1, how='all', inplace=True)
        df.dropna(how='all', inplace=True)


        # Set display option to show all columns
        pd.set_option('display.max_columns', None)
        
        dfs.append(df.reset_index(drop=True))  # Reset index of each DataFrame
        
    return dfs

# Example usage:
clean_dfs = clean_data(table_list)


In [61]:
len(clean_dfs)

7

### This is how each df looks like now


In [62]:
clean_dfs[0]

,Group Name:,Group Number(or TBD),Request Date:,Effective Date or NSB:,Benefits Status,Plan Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product,Access,Network,PCP/Specialist OV Copay,ER Cost Share,Hospital Cost Share,IN Deductible,IN Coinsurance %,IN Coinsurance Limit,IN Out of Pocket Maximum,OON Deductible,OON Coinsurance %,OON Coinsurance Limit,OON Out of Pocket Maximum,MNS ID1
0,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,nan,23,NJ,Large,HMO,Non Gated,Freedom,30/45,150,IP: 0 after deductible OP: 0 after deductible,2500/5000,100,3000/6000,3000/6001,2500/5002,92,32000/50002,32000/50002,NaN
1,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,nan,25,NJ,Large,HMO,Non Gated,Freedom,30/50,160,IP: 0 after deductible OP: 0 after deductible,2500/5000,90,32000/50000,32000/50000,2500/5003,93,32000/50003,32000/50003,NaN
2,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,nan,45,NJ,Large,HMO,Non Gated,Freedom,0/0,nan,IP: 0 after deductible OP: 0 after deductible,2500/5001,91,32000/50001,32000/50001,2500/5004,94,32000/50004,32000/50004,NaN
3,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,30/55,nan,IP: 0 after deductible OP: 0 after deductible,2500/5002,92,32000/50002,32000/50002,2500/5002,92,32000/50002,32000/50002,MNS0000025
4,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,30/60,nan,IP: 0 after deductible OP: 0 after deductible,2500/5003,93,32000/50003,32000/50003,2500/5003,93,32000/50003,32000/50003,MNS0000026
5,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,0/1,nan,IP: 0 after deductible OP: 0 after deductible,2500/5004,94,32000/50004,32000/50004,2500/5004,94,32000/50004,32000/50004,MNS0000027


## FUNCTION TO GET MNS IDS

In [59]:
import pandas as pd
import re

def fetch_mns_ids(string):
    mns_ids = re.findall(r'MNS\D*\s*\w+', string)
    standard_mns_ids=[]
    for mns_id in mns_ids:
        mns_id = re.sub(r'MNS\s*', 'MNS', mns_id)
        
        numeric_match = re.search(r'MNS\D*(\d+)', mns_id)
        if numeric_match:
            digits = numeric_match.group(1)
            # Ensure the number of digits is 7
            digits = digits.zfill(7)
            # Reconstruct the MNS ID with padded zeros
            mns_id = 'MNS' + digits
            standard_mns_ids.append(mns_id)
        else:
            standard_mns_ids.append('')
    
    return standard_mns_ids



def get_mns(df):
    df.fillna('nan',inplace=True)
# Loop through each column
    for column in df.columns:
        # Check if 'MNS' is present anywhere in the column
        if df[column].astype(str).str.contains('MNS').any():
            # Apply the function to extract MNS IDs
            mns_ids = df[column].apply(fetch_mns_ids)

    df['MNS IDs']=mns_ids
    max_mns_ids_count = max(len(ids) for ids in mns_ids)
    padded_mns_ids = [ids + [np.NaN] * (max_mns_ids_count - len(ids)) for ids in mns_ids]
    for i in range(max_mns_ids_count):
        df[f'MNS ID{i+1}'] = [ids[i] if ids and len(ids) > i else None for ids in padded_mns_ids]
    

    df.drop(columns='MNS IDs',inplace=True)

    return df

In [63]:
get_mns(clean_dfs[0])

,Group Name:,Group Number(or TBD),Request Date:,Effective Date or NSB:,Benefits Status,Plan Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product,Access,Network,PCP/Specialist OV Copay,ER Cost Share,Hospital Cost Share,IN Deductible,IN Coinsurance %,IN Coinsurance Limit,IN Out of Pocket Maximum,OON Deductible,OON Coinsurance %,OON Coinsurance Limit,OON Out of Pocket Maximum,MNS ID1
0,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,nan,23,NJ,Large,HMO,Non Gated,Freedom,30/45,150,IP: 0 after deductible OP: 0 after deductible,2500/5000,100,3000/6000,3000/6001,2500/5002,92,32000/50002,32000/50002,NaN
1,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,nan,25,NJ,Large,HMO,Non Gated,Freedom,30/50,160,IP: 0 after deductible OP: 0 after deductible,2500/5000,90,32000/50000,32000/50000,2500/5003,93,32000/50003,32000/50003,NaN
2,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,nan,45,NJ,Large,HMO,Non Gated,Freedom,0/0,nan,IP: 0 after deductible OP: 0 after deductible,2500/5001,91,32000/50001,32000/50001,2500/5004,94,32000/50004,32000/50004,NaN
3,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,30/55,nan,IP: 0 after deductible OP: 0 after deductible,2500/5002,92,32000/50002,32000/50002,2500/5002,92,32000/50002,32000/50002,MNS0000025
4,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,30/60,nan,IP: 0 after deductible OP: 0 after deductible,2500/5003,93,32000/50003,32000/50003,2500/5003,93,32000/50003,32000/50003,MNS0000026
5,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,0/1,nan,IP: 0 after deductible OP: 0 after deductible,2500/5004,94,32000/50004,32000/50004,2500/5004,94,32000/50004,32000/50004,MNS0000027


## Concatenating All the dfs

In [83]:
def combine_data(clean_dfs):
    concatenated_df = pd.concat(clean_dfs, ignore_index=True)
    return concatenated_df


# Example usage:
final_df = combine_data(clean_dfs)
final_df

,Group Name:,Group Number(or TBD),Request Date:,Effective Date or NSB:,Benefits Status,Plan Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product,Access,Network,PCP/Specialist OV Copay,ER Cost Share,Hospital Cost Share,IN Deductible,IN Coinsurance %,IN Coinsurance Limit,IN Out of Pocket Maximum,OON Deductible,OON Coinsurance %,OON Coinsurance Limit,OON Out of Pocket Maximum
0,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,NaN,23,NJ,Large,HMO,Non Gated,Freedom,30/45,150,IP: 0 after deductible OP: 0 after deductible,2500/5000,100,3000/6000,3000/6001,2500/5002,92,32000/50002,32000/50002
1,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,NaN,25,NJ,Large,HMO,Non Gated,Freedom,30/50,160,IP: 0 after deductible OP: 0 after deductible,2500/5000,90,32000/50000,32000/50000,2500/5003,93,32000/50003,32000/50003
2,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,NaN,45,NJ,Large,HMO,Non Gated,Freedom,0/0,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5001,91,32000/50001,32000/50001,2500/5004,94,32000/50004,32000/50004
3,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,30/55,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5002,92,32000/50002,32000/50002,2500/5002,92,32000/50002,32000/50002
4,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,30/60,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5003,93,32000/50003,32000/50003,2500/5003,93,32000/50003,32000/50003
5,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,0/1,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5004,94,32000/50004,32000/50004,2500/5004,94,32000/50004,32000/50004
6,Tienam Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,NaN,23,NJ,Large,HMO,Non Gated,Freedom,30/45,150,IP: 0 after deductible OP: 0 after deductible,2500/5000,100,3000/6000,3000/6001,2500/5002,92,32000/50002,32000/50002
7,Tienam Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,NaN,25,NJ,Large,HMO,Non Gated,Freedom,30/50,160,IP: 0 after deductible OP: 0 after deductible,2500/5000,90,32000/50000,32000/50000,2500/5003,93,32000/50003,32000/50003
8,Tienam Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,NaN,45,NJ,Large,HMO,Non Gated,Freedom,0/0,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5001,91,32000/50001,32000/50001,2500/5004,94,32000/50004,32000/50004
9,Tienam Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,30/55,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5002,92,32000/50002,32000/50002,2500/5002,92,32000/50002,32000/50002


### Notes:

- We need to consider cases where tables are arranged top to bottom. Presently the `separate_table` function only deals with tables arranged left to right

- We need to include the logic to get rid additional/irrelevant texts present in the excel sheet above or below the tables

- Some sheets contain 'Corrected Values', written next to the incorrect values which need to be considered.
